In [1]:
import torch
import pandas as pd
from torch.optim.adam import Adam
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings,  TransformerWordEmbeddings, TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

# load data

In [2]:
# this is the folder in which train, test and dev files reside
data_folder = './dataset/Spooky Author Identification/split/'

# column format indicating which columns hold the text and label(s)
column_name_map = {2: "text", 3: "label"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter=',',    # tab-separated files
) 
label_dict = corpus.make_label_dictionary()

2020-10-22 01:57:39,711 Reading data from dataset/Spooky Author Identification/split
2020-10-22 01:57:39,712 Train: dataset/Spooky Author Identification/split/train.csv
2020-10-22 01:57:39,712 Dev: dataset/Spooky Author Identification/split/dev.csv
2020-10-22 01:57:39,713 Test: dataset/Spooky Author Identification/split/test.csv
2020-10-22 01:57:39,767 Computing label dictionary. Progress:


100%|██████████| 15668/15668 [00:02<00:00, 5498.08it/s]

2020-10-22 01:57:42,736 [b'EAP', b'MWS', b'HPL']


In [3]:
print(corpus.obtain_statistics())

{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 11762,
        "number_of_documents_per_class": {
            "EAP": 4780,
            "MWS": 3642,
            "HPL": 3340
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 354144,
            "min": 4,
            "max": 446,
            "avg": 30.109165107974835
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 3906,
        "number_of_documents_per_class": {
            "EAP": 1554,
            "HPL": 1142,
            "MWS": 1210
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 120056,
            "min": 4,
            "max": 875,
            "avg": 30.736303123399896
        }
    },
    "DEV": {
        "dataset": "DEV",
        "total_number_of_documents": 3911,
        "number_of_documents_per_class": {
            "HPL": 1153,
            "EAP": 15

# glove embeddings

In [36]:
word_embeddings = [WordEmbeddings('glove')]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=256)

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

trainer = ModelTrainer(classifier, corpus)

trainer.train('classifiers/spooky_authorship_classifier_glove',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

2020-10-21 23:29:36,541 ----------------------------------------------------------------------------------------------------
2020-10-21 23:29:36,541 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
    )
    (word_reprojection_map): Linear(in_features=100, out_features=100, bias=True)
    (rnn): GRU(100, 256, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=256, out_features=3, bias=True)
  (loss_function): CrossEntropyLoss()
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-10-21 23:29:36,542 ----------------------------------------------------------------------------------------------------
2020-10-21 23:29:36,542 Corpus: "Corpus: 11762 train + 3911 dev + 3906 test sentences"
2020-10-21 23:29:36,543 ----------------------------------------------------------------------------------------------------
2020-10-21 23:2

2020-10-21 23:31:47,286 epoch 5 - iter 144/368 - loss 0.93159901 - samples/sec: 738.12 - lr: 0.100000
2020-10-21 23:31:48,814 epoch 5 - iter 180/368 - loss 0.93545088 - samples/sec: 787.14 - lr: 0.100000
2020-10-21 23:31:52,467 epoch 5 - iter 216/368 - loss 0.93576069 - samples/sec: 786.86 - lr: 0.100000
2020-10-21 23:31:53,979 epoch 5 - iter 252/368 - loss 0.93436251 - samples/sec: 794.39 - lr: 0.100000
2020-10-21 23:31:55,521 epoch 5 - iter 288/368 - loss 0.93376560 - samples/sec: 777.09 - lr: 0.100000
2020-10-21 23:31:57,153 epoch 5 - iter 324/368 - loss 0.93307721 - samples/sec: 734.47 - lr: 0.100000
2020-10-21 23:31:58,769 epoch 5 - iter 360/368 - loss 0.92926995 - samples/sec: 742.29 - lr: 0.100000
2020-10-21 23:31:59,223 ----------------------------------------------------------------------------------------------------
2020-10-21 23:31:59,224 EPOCH 5 done: loss 0.9314 - lr 0.1000000
2020-10-21 23:32:07,426 DEV : loss 0.8543479442596436 - score 0.6339
2020-10-21 23:32:09,326 BAD

2020-10-21 23:34:36,552 DEV : loss 0.8141556978225708 - score 0.6446
2020-10-21 23:34:38,455 BAD EPOCHS (no improvement): 1
2020-10-21 23:34:38,456 ----------------------------------------------------------------------------------------------------
2020-10-21 23:34:40,530 epoch 11 - iter 36/368 - loss 0.85544606 - samples/sec: 638.11 - lr: 0.100000
2020-10-21 23:34:44,429 epoch 11 - iter 72/368 - loss 0.84553569 - samples/sec: 705.24 - lr: 0.100000
2020-10-21 23:34:46,108 epoch 11 - iter 108/368 - loss 0.83701550 - samples/sec: 714.98 - lr: 0.100000
2020-10-21 23:34:47,778 epoch 11 - iter 144/368 - loss 0.82869205 - samples/sec: 718.05 - lr: 0.100000
2020-10-21 23:34:49,350 epoch 11 - iter 180/368 - loss 0.82934599 - samples/sec: 762.89 - lr: 0.100000
2020-10-21 23:34:50,991 epoch 11 - iter 216/368 - loss 0.82982853 - samples/sec: 734.55 - lr: 0.100000
2020-10-21 23:34:52,578 epoch 11 - iter 252/368 - loss 0.82439812 - samples/sec: 757.42 - lr: 0.100000
2020-10-21 23:34:54,232 epoch 11

2020-10-21 23:37:26,390 epoch 16 - iter 252/368 - loss 0.78221223 - samples/sec: 792.90 - lr: 0.100000
2020-10-21 23:37:27,960 epoch 16 - iter 288/368 - loss 0.78330726 - samples/sec: 765.72 - lr: 0.100000
2020-10-21 23:37:29,714 epoch 16 - iter 324/368 - loss 0.78588932 - samples/sec: 684.91 - lr: 0.100000
2020-10-21 23:37:31,391 epoch 16 - iter 360/368 - loss 0.78471214 - samples/sec: 713.99 - lr: 0.100000
2020-10-21 23:37:31,778 ----------------------------------------------------------------------------------------------------
2020-10-21 23:37:31,778 EPOCH 16 done: loss 0.7848 - lr 0.1000000
2020-10-21 23:37:40,544 DEV : loss 0.6886066794395447 - score 0.7111
2020-10-21 23:37:42,504 BAD EPOCHS (no improvement): 0
saving best model
2020-10-21 23:37:44,207 ----------------------------------------------------------------------------------------------------
2020-10-21 23:37:46,457 epoch 17 - iter 36/368 - loss 0.77470153 - samples/sec: 597.24 - lr: 0.100000
2020-10-21 23:37:48,294 epoc

2020-10-21 23:40:20,836 epoch 22 - iter 36/368 - loss 0.69647878 - samples/sec: 645.69 - lr: 0.100000
2020-10-21 23:40:22,510 epoch 22 - iter 72/368 - loss 0.72251955 - samples/sec: 721.13 - lr: 0.100000
2020-10-21 23:40:24,171 epoch 22 - iter 108/368 - loss 0.73687818 - samples/sec: 727.06 - lr: 0.100000
2020-10-21 23:40:25,857 epoch 22 - iter 144/368 - loss 0.73680936 - samples/sec: 717.18 - lr: 0.100000
2020-10-21 23:40:27,567 epoch 22 - iter 180/368 - loss 0.73450513 - samples/sec: 702.94 - lr: 0.100000
2020-10-21 23:40:29,268 epoch 22 - iter 216/368 - loss 0.73652112 - samples/sec: 708.23 - lr: 0.100000
2020-10-21 23:40:33,613 epoch 22 - iter 252/368 - loss 0.73656856 - samples/sec: 635.70 - lr: 0.100000
2020-10-21 23:40:35,341 epoch 22 - iter 288/368 - loss 0.73935149 - samples/sec: 694.91 - lr: 0.100000
2020-10-21 23:40:37,051 epoch 22 - iter 324/368 - loss 0.73551780 - samples/sec: 701.49 - lr: 0.100000
2020-10-21 23:40:38,779 epoch 22 - iter 360/368 - loss 0.73459725 - samples

...

2020-10-22 00:20:04,646 epoch 100 - iter 216/368 - loss 0.48473669 - samples/sec: 751.73 - lr: 0.000781
2020-10-22 00:20:06,271 epoch 100 - iter 252/368 - loss 0.47904748 - samples/sec: 739.84 - lr: 0.000781
2020-10-22 00:20:07,892 epoch 100 - iter 288/368 - loss 0.47648268 - samples/sec: 740.50 - lr: 0.000781
2020-10-22 00:20:09,504 epoch 100 - iter 324/368 - loss 0.47251173 - samples/sec: 748.58 - lr: 0.000781
2020-10-22 00:20:11,072 epoch 100 - iter 360/368 - loss 0.47580548 - samples/sec: 768.06 - lr: 0.000781
2020-10-22 00:20:13,558 ----------------------------------------------------------------------------------------------------
2020-10-22 00:20:13,559 EPOCH 100 done: loss 0.4759 - lr 0.0007813
2020-10-22 00:20:19,744 DEV : loss 0.5817830562591553 - score 0.7827
2020-10-22 00:20:21,669 BAD EPOCHS (no improvement): 1
2020-10-22 00:20:21,670 ----------------------------------------------------------------------------------------------------
2020-10-22 00:20:23,853 epoch 101 - ite

2020-10-22 00:22:52,164 ----------------------------------------------------------------------------------------------------
2020-10-22 00:22:54,361 epoch 106 - iter 36/368 - loss 0.44022814 - samples/sec: 613.12 - lr: 0.000391
2020-10-22 00:22:56,168 epoch 106 - iter 72/368 - loss 0.45736753 - samples/sec: 665.78 - lr: 0.000391
2020-10-22 00:22:57,821 epoch 106 - iter 108/368 - loss 0.47360443 - samples/sec: 726.85 - lr: 0.000391
2020-10-22 00:22:59,539 epoch 106 - iter 144/368 - loss 0.47645261 - samples/sec: 698.98 - lr: 0.000391
2020-10-22 00:23:03,458 epoch 106 - iter 180/368 - loss 0.47635900 - samples/sec: 299.58 - lr: 0.000391
2020-10-22 00:23:05,154 epoch 106 - iter 216/368 - loss 0.47604843 - samples/sec: 708.66 - lr: 0.000391
2020-10-22 00:23:06,812 epoch 106 - iter 252/368 - loss 0.47168294 - samples/sec: 724.02 - lr: 0.000391
2020-10-22 00:23:08,457 epoch 106 - iter 288/368 - loss 0.47200430 - samples/sec: 728.51 - lr: 0.000391
2020-10-22 00:23:10,032 epoch 106 - iter 324/

2020-10-22 00:25:46,458 epoch 111 - iter 288/368 - loss 0.48230134 - samples/sec: 807.66 - lr: 0.000391
2020-10-22 00:25:50,079 epoch 111 - iter 324/368 - loss 0.48483508 - samples/sec: 771.85 - lr: 0.000391
2020-10-22 00:25:51,716 epoch 111 - iter 360/368 - loss 0.48844734 - samples/sec: 732.82 - lr: 0.000391
2020-10-22 00:25:52,182 ----------------------------------------------------------------------------------------------------
2020-10-22 00:25:52,183 EPOCH 111 done: loss 0.4878 - lr 0.0003906
2020-10-22 00:25:58,499 DEV : loss 0.5836184620857239 - score 0.7811
Epoch   111: reducing learning rate of group 0 to 1.9531e-04.
2020-10-22 00:26:00,418 BAD EPOCHS (no improvement): 6
2020-10-22 00:26:00,419 ----------------------------------------------------------------------------------------------------
2020-10-22 00:26:02,597 epoch 112 - iter 36/368 - loss 0.45687327 - samples/sec: 613.19 - lr: 0.000195
2020-10-22 00:26:04,337 epoch 112 - iter 72/368 - loss 0.46682370 - samples/sec: 6

2020-10-22 00:28:32,591 epoch 117 - iter 36/368 - loss 0.44484593 - samples/sec: 594.28 - lr: 0.000195
2020-10-22 00:28:34,322 epoch 117 - iter 72/368 - loss 0.45819084 - samples/sec: 697.76 - lr: 0.000195
2020-10-22 00:28:35,900 epoch 117 - iter 108/368 - loss 0.46395852 - samples/sec: 765.60 - lr: 0.000195
2020-10-22 00:28:37,548 epoch 117 - iter 144/368 - loss 0.46154677 - samples/sec: 737.57 - lr: 0.000195
2020-10-22 00:28:39,216 epoch 117 - iter 180/368 - loss 0.46453326 - samples/sec: 723.39 - lr: 0.000195
2020-10-22 00:28:40,892 epoch 117 - iter 216/368 - loss 0.46760760 - samples/sec: 719.47 - lr: 0.000195
2020-10-22 00:28:44,668 epoch 117 - iter 252/368 - loss 0.47318098 - samples/sec: 752.29 - lr: 0.000195
2020-10-22 00:28:46,412 epoch 117 - iter 288/368 - loss 0.47503049 - samples/sec: 688.98 - lr: 0.000195
2020-10-22 00:28:48,096 epoch 117 - iter 324/368 - loss 0.47390440 - samples/sec: 715.52 - lr: 0.000195
2020-10-22 00:28:49,677 epoch 117 - iter 360/368 - loss 0.47584448

{'test_score': 0.7824,
 'dev_score_history': [0.5098,
  0.4845,
  0.5687,
  0.3728,
  0.6339,
  0.6249,
  0.6349,
  0.6305,
  0.6707,
  0.6446,
  0.686,
  0.6911,
  0.6137,
  0.5566,
  0.6916,
  0.7111,
  0.6796,
  0.7095,
  0.7236,
  0.6898,
  0.674,
  0.7185,
  0.7095,
  0.732,
  0.7037,
  0.6988,
  0.6883,
  0.6975,
  0.7504,
  0.7443,
  0.7274,
  0.7558,
  0.754,
  0.6824,
  0.7072,
  0.7369,
  0.6939,
  0.7481,
  0.763,
  0.7466,
  0.7515,
  0.7579,
  0.7589,
  0.7553,
  0.7512,
  0.7645,
  0.7694,
  0.7699,
  0.753,
  0.7655,
  0.7788,
  0.7627,
  0.7706,
  0.7612,
  0.775,
  0.775,
  0.7694,
  0.763,
  0.7755,
  0.7717,
  0.7686,
  0.7747,
  0.7691,
  0.7722,
  0.7691,
  0.7719,
  0.7758,
  0.7776,
  0.777,
  0.7814,
  0.7752,
  0.7742,
  0.7776,
  0.7737,
  0.7816,
  0.7783,
  0.7755,
  0.7816,
  0.7801,
  0.7806,
  0.7722,
  0.7786,
  0.7827,
  0.7801,
  0.7819,
  0.7819,
  0.7755,
  0.7801,
  0.7768,
  0.7837,
  0.7806,
  0.7811,
  0.7842,
  0.7814,
  0.7832,
  0.7829,
  0.78

# glove + for/backward flair embeddings

In [4]:
torch.cuda.empty_cache()
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward'), FlairEmbeddings('news-backward')]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=256)

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

trainer = ModelTrainer(classifier, corpus)

trainer.train('classifiers/spooky_authorship_classifier_glove&news-fbflair',
              learning_rate=0.1,
              mini_batch_size=16,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

2020-10-22 05:39:29,219 ----------------------------------------------------------------------------------------------------
2020-10-22 05:39:29,220 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=4196, out_features=4196, bias=True)
    (rnn): GRU(4196, 256, 

2020-10-22 05:55:14,081 EPOCH 4 done: loss 0.7073 - lr 0.1000000
2020-10-22 05:56:14,354 DEV : loss 1.021492600440979 - score 0.5883
2020-10-22 05:56:16,260 BAD EPOCHS (no improvement): 1
2020-10-22 05:56:16,261 ----------------------------------------------------------------------------------------------------
2020-10-22 05:56:36,800 epoch 5 - iter 73/736 - loss 0.67823635 - samples/sec: 58.21 - lr: 0.100000
2020-10-22 05:56:55,847 epoch 5 - iter 146/736 - loss 0.70018424 - samples/sec: 62.51 - lr: 0.100000
2020-10-22 05:57:14,514 epoch 5 - iter 219/736 - loss 0.68725666 - samples/sec: 63.77 - lr: 0.100000
2020-10-22 05:57:33,527 epoch 5 - iter 292/736 - loss 0.66753892 - samples/sec: 61.70 - lr: 0.100000
2020-10-22 05:57:51,318 epoch 5 - iter 365/736 - loss 0.66906751 - samples/sec: 67.03 - lr: 0.100000
2020-10-22 05:58:10,088 epoch 5 - iter 438/736 - loss 0.67636088 - samples/sec: 63.49 - lr: 0.100000
2020-10-22 05:58:28,429 epoch 5 - iter 511/736 - loss 0.68132020 - samples/sec: 64

2020-10-22 06:19:22,747 epoch 10 - iter 511/736 - loss 0.56972459 - samples/sec: 64.91 - lr: 0.100000
2020-10-22 06:19:41,728 epoch 10 - iter 584/736 - loss 0.56812148 - samples/sec: 61.84 - lr: 0.100000
2020-10-22 06:20:00,440 epoch 10 - iter 657/736 - loss 0.57327429 - samples/sec: 63.72 - lr: 0.100000
2020-10-22 06:20:18,870 epoch 10 - iter 730/736 - loss 0.57654182 - samples/sec: 64.66 - lr: 0.100000
2020-10-22 06:20:20,272 ----------------------------------------------------------------------------------------------------
2020-10-22 06:20:20,273 EPOCH 10 done: loss 0.5753 - lr 0.1000000
2020-10-22 06:21:20,555 DEV : loss 0.5725451707839966 - score 0.7683
2020-10-22 06:21:22,450 BAD EPOCHS (no improvement): 1
2020-10-22 06:21:22,451 ----------------------------------------------------------------------------------------------------
2020-10-22 06:21:41,893 epoch 11 - iter 73/736 - loss 0.48916072 - samples/sec: 61.57 - lr: 0.100000
2020-10-22 06:22:00,466 epoch 11 - iter 146/736 - l

2020-10-22 06:42:54,838 epoch 16 - iter 146/736 - loss 0.48044030 - samples/sec: 62.06 - lr: 0.100000
2020-10-22 06:43:13,309 epoch 16 - iter 219/736 - loss 0.48963711 - samples/sec: 65.41 - lr: 0.100000
2020-10-22 06:43:32,176 epoch 16 - iter 292/736 - loss 0.49337707 - samples/sec: 63.12 - lr: 0.100000
2020-10-22 06:43:49,431 epoch 16 - iter 365/736 - loss 0.48550541 - samples/sec: 69.08 - lr: 0.100000
2020-10-22 06:44:08,423 epoch 16 - iter 438/736 - loss 0.48504848 - samples/sec: 62.65 - lr: 0.100000
2020-10-22 06:44:26,719 epoch 16 - iter 511/736 - loss 0.48881208 - samples/sec: 64.10 - lr: 0.100000
2020-10-22 06:44:45,827 epoch 16 - iter 584/736 - loss 0.48812099 - samples/sec: 62.30 - lr: 0.100000
2020-10-22 06:45:04,432 epoch 16 - iter 657/736 - loss 0.49027505 - samples/sec: 64.02 - lr: 0.100000
2020-10-22 06:45:24,350 epoch 16 - iter 730/736 - loss 0.48663807 - samples/sec: 60.54 - lr: 0.100000
2020-10-22 06:45:25,830 ----------------------------------------------------------

2020-10-22 07:06:18,648 ----------------------------------------------------------------------------------------------------
2020-10-22 07:06:18,648 EPOCH 21 done: loss 0.4434 - lr 0.1000000
2020-10-22 07:07:18,931 DEV : loss 0.5002753734588623 - score 0.8057
2020-10-22 07:07:20,837 BAD EPOCHS (no improvement): 0
saving best model
2020-10-22 07:07:22,824 ----------------------------------------------------------------------------------------------------
2020-10-22 07:07:41,767 epoch 22 - iter 73/736 - loss 0.38669641 - samples/sec: 64.27 - lr: 0.100000
2020-10-22 07:07:59,639 epoch 22 - iter 146/736 - loss 0.40790390 - samples/sec: 66.63 - lr: 0.100000
2020-10-22 07:08:19,275 epoch 22 - iter 219/736 - loss 0.42489670 - samples/sec: 59.77 - lr: 0.100000
2020-10-22 07:08:38,260 epoch 22 - iter 292/736 - loss 0.41745021 - samples/sec: 62.65 - lr: 0.100000
2020-10-22 07:08:56,715 epoch 22 - iter 365/736 - loss 0.42424755 - samples/sec: 64.51 - lr: 0.100000
2020-10-22 07:09:15,882 epoch 22 

...

2020-10-22 12:47:14,543 epoch 100 - iter 365/736 - loss 0.07548293 - samples/sec: 64.33 - lr: 0.000781
2020-10-22 12:47:33,489 epoch 100 - iter 438/736 - loss 0.07493269 - samples/sec: 61.91 - lr: 0.000781
2020-10-22 12:47:53,510 epoch 100 - iter 511/736 - loss 0.07719064 - samples/sec: 60.26 - lr: 0.000781
2020-10-22 12:48:12,984 epoch 100 - iter 584/736 - loss 0.07749062 - samples/sec: 61.04 - lr: 0.000781
2020-10-22 12:48:30,609 epoch 100 - iter 657/736 - loss 0.07714540 - samples/sec: 66.54 - lr: 0.000781
2020-10-22 12:48:48,755 epoch 100 - iter 730/736 - loss 0.07807375 - samples/sec: 65.66 - lr: 0.000781
2020-10-22 12:48:50,392 ----------------------------------------------------------------------------------------------------
2020-10-22 12:48:50,393 EPOCH 100 done: loss 0.0782 - lr 0.0007813
2020-10-22 12:49:50,780 DEV : loss 0.7138060927391052 - score 0.8448
2020-10-22 12:49:52,667 BAD EPOCHS (no improvement): 1
2020-10-22 12:49:52,667 ------------------------------------------

Epoch   105: reducing learning rate of group 0 to 3.9063e-04.
2020-10-22 13:10:47,190 BAD EPOCHS (no improvement): 6
2020-10-22 13:10:47,191 ----------------------------------------------------------------------------------------------------
2020-10-22 13:11:05,877 epoch 106 - iter 73/736 - loss 0.06386210 - samples/sec: 64.16 - lr: 0.000391
2020-10-22 13:11:24,167 epoch 106 - iter 146/736 - loss 0.06407240 - samples/sec: 65.09 - lr: 0.000391
2020-10-22 13:11:41,890 epoch 106 - iter 219/736 - loss 0.06618554 - samples/sec: 66.18 - lr: 0.000391
2020-10-22 13:11:59,351 epoch 106 - iter 292/736 - loss 0.07083982 - samples/sec: 68.22 - lr: 0.000391
2020-10-22 13:12:18,912 epoch 106 - iter 365/736 - loss 0.07453130 - samples/sec: 60.78 - lr: 0.000391
2020-10-22 13:12:36,366 epoch 106 - iter 438/736 - loss 0.07466258 - samples/sec: 67.22 - lr: 0.000391
2020-10-22 13:12:56,661 epoch 106 - iter 511/736 - loss 0.07369853 - samples/sec: 58.56 - lr: 0.000391
2020-10-22 13:13:15,057 epoch 106 - it

2020-10-22 13:33:58,783 epoch 111 - iter 511/736 - loss 0.07121711 - samples/sec: 62.55 - lr: 0.000391
2020-10-22 13:34:18,387 epoch 111 - iter 584/736 - loss 0.07290009 - samples/sec: 60.75 - lr: 0.000391
2020-10-22 13:34:38,230 epoch 111 - iter 657/736 - loss 0.07342869 - samples/sec: 60.82 - lr: 0.000391
2020-10-22 13:34:55,461 epoch 111 - iter 730/736 - loss 0.07343918 - samples/sec: 68.20 - lr: 0.000391
2020-10-22 13:34:56,786 ----------------------------------------------------------------------------------------------------
2020-10-22 13:34:56,787 EPOCH 111 done: loss 0.0734 - lr 0.0003906
2020-10-22 13:35:57,624 DEV : loss 0.7158766388893127 - score 0.8453
Epoch   111: reducing learning rate of group 0 to 1.9531e-04.
2020-10-22 13:35:59,518 BAD EPOCHS (no improvement): 6
2020-10-22 13:35:59,519 ----------------------------------------------------------------------------------------------------
2020-10-22 13:36:18,497 epoch 112 - iter 73/736 - loss 0.06487043 - samples/sec: 62.2

2020-10-22 13:57:15,598 epoch 117 - iter 73/736 - loss 0.07890967 - samples/sec: 61.11 - lr: 0.000195
2020-10-22 13:57:33,078 epoch 117 - iter 146/736 - loss 0.09450423 - samples/sec: 67.14 - lr: 0.000195
2020-10-22 13:57:52,854 epoch 117 - iter 219/736 - loss 0.08524608 - samples/sec: 59.35 - lr: 0.000195
2020-10-22 13:58:11,174 epoch 117 - iter 292/736 - loss 0.08235634 - samples/sec: 65.02 - lr: 0.000195
2020-10-22 13:58:29,317 epoch 117 - iter 365/736 - loss 0.07857722 - samples/sec: 65.77 - lr: 0.000195
2020-10-22 13:58:48,658 epoch 117 - iter 438/736 - loss 0.07764626 - samples/sec: 61.59 - lr: 0.000195
2020-10-22 13:59:07,196 epoch 117 - iter 511/736 - loss 0.07607066 - samples/sec: 64.25 - lr: 0.000195
2020-10-22 13:59:26,489 epoch 117 - iter 584/736 - loss 0.07461531 - samples/sec: 60.79 - lr: 0.000195
2020-10-22 13:59:45,802 epoch 117 - iter 657/736 - loss 0.07294729 - samples/sec: 61.57 - lr: 0.000195
2020-10-22 14:00:04,218 epoch 117 - iter 730/736 - loss 0.07321712 - sampl

{'test_score': 0.8479,
 'dev_score_history': [0.5313,
  0.5973,
  0.6129,
  0.5883,
  0.7433,
  0.7686,
  0.7246,
  0.7428,
  0.776,
  0.7683,
  0.764,
  0.7891,
  0.7415,
  0.7954,
  0.7724,
  0.7919,
  0.7384,
  0.7908,
  0.7768,
  0.7977,
  0.8057,
  0.7763,
  0.8113,
  0.798,
  0.7949,
  0.787,
  0.8072,
  0.8021,
  0.785,
  0.8179,
  0.8192,
  0.8259,
  0.8307,
  0.8213,
  0.8151,
  0.8266,
  0.832,
  0.8249,
  0.8297,
  0.8346,
  0.8266,
  0.8289,
  0.8233,
  0.8295,
  0.8249,
  0.8246,
  0.8292,
  0.8333,
  0.8346,
  0.8366,
  0.8333,
  0.8394,
  0.8338,
  0.8346,
  0.8338,
  0.8284,
  0.8394,
  0.8335,
  0.8415,
  0.8394,
  0.8402,
  0.8394,
  0.8399,
  0.843,
  0.8425,
  0.8399,
  0.8374,
  0.844,
  0.8405,
  0.842,
  0.8387,
  0.8394,
  0.842,
  0.8381,
  0.8422,
  0.842,
  0.8407,
  0.8417,
  0.8402,
  0.8415,
  0.841,
  0.8435,
  0.8407,
  0.8438,
  0.8435,
  0.8402,
  0.8453,
  0.8428,
  0.8458,
  0.844,
  0.8445,
  0.8438,
  0.8458,
  0.8458,
  0.8448,
  0.8456,
  0.8435,

# bert embeddings

In [5]:
torch.cuda.empty_cache()

word_embeddings = [TransformerWordEmbeddings('bert-base-uncased')]

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=256)

# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

# 7. start the training
trainer.train('classifiers/spooky_authorship_classifier_bert',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

2020-10-22 14:02:09,538 ----------------------------------------------------------------------------------------------------
2020-10-22 14:02:09,540 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): TransformerWordEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Line

2020-10-22 14:02:09,541 ----------------------------------------------------------------------------------------------------
2020-10-22 14:02:09,541 Corpus: "Corpus: 11762 train + 3911 dev + 3906 test sentences"
2020-10-22 14:02:09,542 ----------------------------------------------------------------------------------------------------
2020-10-22 14:02:09,542 Parameters:
2020-10-22 14:02:09,542  - learning_rate: "0.1"
2020-10-22 14:02:09,543  - mini_batch_size: "32"
2020-10-22 14:02:09,543  - patience: "5"
2020-10-22 14:02:09,543  - anneal_factor: "0.5"
2020-10-22 14:02:09,544  - max_epochs: "150"
2020-10-22 14:02:09,544  - shuffle: "True"
2020-10-22 14:02:09,544  - train_with_dev: "False"
2020-10-22 14:02:09,544  - batch_growth_annealing: "False"
2020-10-22 14:02:09,545 ----------------------------------------------------------------------------------------------------
2020-10-22 14:02:09,545 Model training base path: "classifiers/spooky_authorship_classifier_bert"
2020-10-22 14:02:09,

2020-10-22 14:19:36,282 ----------------------------------------------------------------------------------------------------
2020-10-22 14:19:36,282 EPOCH 5 done: loss 0.5798 - lr 0.1000000
2020-10-22 14:20:23,867 DEV : loss 0.7465242147445679 - score 0.6891
2020-10-22 14:20:25,766 BAD EPOCHS (no improvement): 2
2020-10-22 14:20:25,767 ----------------------------------------------------------------------------------------------------
2020-10-22 14:20:42,654 epoch 6 - iter 36/368 - loss 0.51457353 - samples/sec: 70.37 - lr: 0.100000
2020-10-22 14:20:59,459 epoch 6 - iter 72/368 - loss 0.51202875 - samples/sec: 69.80 - lr: 0.100000
2020-10-22 14:21:16,020 epoch 6 - iter 108/368 - loss 0.53477021 - samples/sec: 70.87 - lr: 0.100000
2020-10-22 14:21:32,424 epoch 6 - iter 144/368 - loss 0.54743409 - samples/sec: 71.59 - lr: 0.100000
2020-10-22 14:21:48,974 epoch 6 - iter 180/368 - loss 0.54946897 - samples/sec: 70.98 - lr: 0.100000
2020-10-22 14:22:05,394 epoch 6 - iter 216/368 - loss 0.55

2020-10-22 14:40:22,904 epoch 11 - iter 216/368 - loss 0.48324032 - samples/sec: 71.90 - lr: 0.100000
2020-10-22 14:40:38,993 epoch 11 - iter 252/368 - loss 0.48514123 - samples/sec: 71.89 - lr: 0.100000
2020-10-22 14:40:55,322 epoch 11 - iter 288/368 - loss 0.49061823 - samples/sec: 71.89 - lr: 0.100000
2020-10-22 14:41:11,747 epoch 11 - iter 324/368 - loss 0.49279711 - samples/sec: 71.46 - lr: 0.100000
2020-10-22 14:41:28,560 epoch 11 - iter 360/368 - loss 0.49154352 - samples/sec: 69.85 - lr: 0.100000
2020-10-22 14:41:32,344 ----------------------------------------------------------------------------------------------------
2020-10-22 14:41:32,345 EPOCH 11 done: loss 0.4918 - lr 0.1000000
2020-10-22 14:42:19,870 DEV : loss 0.5237590074539185 - score 0.7786
2020-10-22 14:42:21,743 BAD EPOCHS (no improvement): 1
2020-10-22 14:42:21,744 ----------------------------------------------------------------------------------------------------
2020-10-22 14:42:38,845 epoch 12 - iter 36/368 - l

2020-10-22 15:00:54,335 epoch 17 - iter 36/368 - loss 0.46092704 - samples/sec: 68.99 - lr: 0.100000
2020-10-22 15:01:10,281 epoch 17 - iter 72/368 - loss 0.47088402 - samples/sec: 72.52 - lr: 0.100000
2020-10-22 15:01:27,102 epoch 17 - iter 108/368 - loss 0.46656210 - samples/sec: 69.79 - lr: 0.100000
2020-10-22 15:01:43,738 epoch 17 - iter 144/368 - loss 0.46607319 - samples/sec: 70.57 - lr: 0.100000
2020-10-22 15:02:00,426 epoch 17 - iter 180/368 - loss 0.46985867 - samples/sec: 70.38 - lr: 0.100000
2020-10-22 15:02:17,203 epoch 17 - iter 216/368 - loss 0.46655500 - samples/sec: 69.98 - lr: 0.100000
2020-10-22 15:02:33,734 epoch 17 - iter 252/368 - loss 0.46984156 - samples/sec: 71.03 - lr: 0.100000
2020-10-22 15:02:50,007 epoch 17 - iter 288/368 - loss 0.46923452 - samples/sec: 71.10 - lr: 0.100000
2020-10-22 15:03:06,502 epoch 17 - iter 324/368 - loss 0.46761320 - samples/sec: 71.21 - lr: 0.100000
2020-10-22 15:03:23,098 epoch 17 - iter 360/368 - loss 0.46933555 - samples/sec: 70.

...

2020-10-22 20:32:44,971 DEV : loss 0.4204995036125183 - score 0.8537
2020-10-22 20:32:46,860 BAD EPOCHS (no improvement): 4
2020-10-22 20:32:46,860 ----------------------------------------------------------------------------------------------------
2020-10-22 20:33:03,822 epoch 107 - iter 36/368 - loss 0.15842249 - samples/sec: 70.19 - lr: 0.001563
2020-10-22 20:33:20,275 epoch 107 - iter 72/368 - loss 0.15417178 - samples/sec: 71.40 - lr: 0.001563
2020-10-22 20:33:36,650 epoch 107 - iter 108/368 - loss 0.15052047 - samples/sec: 71.69 - lr: 0.001563
2020-10-22 20:33:53,246 epoch 107 - iter 144/368 - loss 0.14782753 - samples/sec: 70.73 - lr: 0.001563
2020-10-22 20:34:09,723 epoch 107 - iter 180/368 - loss 0.14994712 - samples/sec: 70.21 - lr: 0.001563
2020-10-22 20:34:26,792 epoch 107 - iter 216/368 - loss 0.15163033 - samples/sec: 68.79 - lr: 0.001563
2020-10-22 20:34:43,803 epoch 107 - iter 252/368 - loss 0.15213532 - samples/sec: 69.02 - lr: 0.001563
2020-10-22 20:35:02,295 epoch 10

2020-10-22 20:53:28,006 epoch 112 - iter 252/368 - loss 0.14145082 - samples/sec: 68.73 - lr: 0.000781
2020-10-22 20:53:44,692 epoch 112 - iter 288/368 - loss 0.14796462 - samples/sec: 70.40 - lr: 0.000781
2020-10-22 20:54:01,488 epoch 112 - iter 324/368 - loss 0.15068915 - samples/sec: 69.86 - lr: 0.000781
2020-10-22 20:54:18,526 epoch 112 - iter 360/368 - loss 0.15049402 - samples/sec: 67.88 - lr: 0.000781
2020-10-22 20:54:22,067 ----------------------------------------------------------------------------------------------------
2020-10-22 20:54:22,068 EPOCH 112 done: loss 0.1499 - lr 0.0007813
2020-10-22 20:55:11,263 DEV : loss 0.42142152786254883 - score 0.8527
2020-10-22 20:55:13,181 BAD EPOCHS (no improvement): 4
2020-10-22 20:55:13,182 ----------------------------------------------------------------------------------------------------
2020-10-22 20:55:30,759 epoch 113 - iter 36/368 - loss 0.19185520 - samples/sec: 67.43 - lr: 0.000781
2020-10-22 20:55:47,602 epoch 113 - iter 72/

2020-10-22 21:14:15,675 epoch 118 - iter 36/368 - loss 0.15156224 - samples/sec: 70.50 - lr: 0.000391
2020-10-22 21:14:32,902 epoch 118 - iter 72/368 - loss 0.14646500 - samples/sec: 67.12 - lr: 0.000391
2020-10-22 21:14:50,330 epoch 118 - iter 108/368 - loss 0.14566728 - samples/sec: 67.39 - lr: 0.000391
2020-10-22 21:15:07,331 epoch 118 - iter 144/368 - loss 0.14783675 - samples/sec: 68.00 - lr: 0.000391
2020-10-22 21:15:24,697 epoch 118 - iter 180/368 - loss 0.15149259 - samples/sec: 67.55 - lr: 0.000391
2020-10-22 21:15:41,564 epoch 118 - iter 216/368 - loss 0.14889265 - samples/sec: 68.59 - lr: 0.000391
2020-10-22 21:15:58,650 epoch 118 - iter 252/368 - loss 0.14959316 - samples/sec: 68.65 - lr: 0.000391
2020-10-22 21:16:15,602 epoch 118 - iter 288/368 - loss 0.14834335 - samples/sec: 68.23 - lr: 0.000391
2020-10-22 21:16:32,556 epoch 118 - iter 324/368 - loss 0.14866673 - samples/sec: 69.23 - lr: 0.000391
2020-10-22 21:16:48,999 epoch 118 - iter 360/368 - loss 0.15060692 - sample

2020-10-22 21:35:04,860 epoch 123 - iter 324/368 - loss 0.14747788 - samples/sec: 69.92 - lr: 0.000195
2020-10-22 21:35:21,868 epoch 123 - iter 360/368 - loss 0.14739360 - samples/sec: 69.00 - lr: 0.000195
2020-10-22 21:35:25,607 ----------------------------------------------------------------------------------------------------
2020-10-22 21:35:25,607 EPOCH 123 done: loss 0.1474 - lr 0.0001953
2020-10-22 21:36:15,522 DEV : loss 0.4210428297519684 - score 0.8522
2020-10-22 21:36:17,402 BAD EPOCHS (no improvement): 3
2020-10-22 21:36:17,403 ----------------------------------------------------------------------------------------------------
2020-10-22 21:36:34,464 epoch 124 - iter 36/368 - loss 0.15370604 - samples/sec: 68.48 - lr: 0.000195
2020-10-22 21:36:51,411 epoch 124 - iter 72/368 - loss 0.14630944 - samples/sec: 69.23 - lr: 0.000195
2020-10-22 21:37:07,873 epoch 124 - iter 108/368 - loss 0.14446120 - samples/sec: 70.28 - lr: 0.000195
2020-10-22 21:37:24,555 epoch 124 - iter 144/3

{'test_score': 0.8495,
 'dev_score_history': [0.7223,
  0.7195,
  0.7901,
  0.7735,
  0.6891,
  0.7216,
  0.7972,
  0.7919,
  0.7921,
  0.8052,
  0.7786,
  0.8052,
  0.7763,
  0.8141,
  0.8151,
  0.8082,
  0.81,
  0.7435,
  0.8141,
  0.8133,
  0.8167,
  0.8062,
  0.7837,
  0.8251,
  0.8192,
  0.8172,
  0.8159,
  0.7783,
  0.8149,
  0.8149,
  0.831,
  0.8274,
  0.8287,
  0.8272,
  0.8348,
  0.8379,
  0.8343,
  0.8289,
  0.8374,
  0.8282,
  0.8256,
  0.8264,
  0.8415,
  0.8369,
  0.8376,
  0.8356,
  0.8417,
  0.8392,
  0.841,
  0.8433,
  0.8381,
  0.8384,
  0.8458,
  0.8438,
  0.8433,
  0.8422,
  0.8456,
  0.8481,
  0.8463,
  0.8479,
  0.8453,
  0.8458,
  0.8502,
  0.8514,
  0.8486,
  0.8443,
  0.843,
  0.8445,
  0.8497,
  0.8512,
  0.8502,
  0.8491,
  0.8517,
  0.8517,
  0.8497,
  0.8502,
  0.8499,
  0.8481,
  0.8504,
  0.8494,
  0.8509,
  0.8486,
  0.8525,
  0.8509,
  0.8553,
  0.8512,
  0.852,
  0.8525,
  0.8545,
  0.8566,
  0.8543,
  0.8507,
  0.8532,
  0.8525,
  0.8512,
  0.852,
  0

# transformer

In [3]:
torch.cuda.empty_cache()

document_embeddings = TransformerDocumentEmbeddings('bert-base-uncased', fine_tune=True)

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

trainer.train('classifiers/spooky_authorship_classifier_transformer',
              learning_rate=3e-5, # use very small learning rate
              mini_batch_size=1,
              mini_batch_chunk_size=4, # optionally set this if transformer is too much for your machine
              max_epochs=50, # terminate after 5 epochs
              )

2020-10-22 01:59:06,967 ----------------------------------------------------------------------------------------------------
2020-10-22 01:59:06,969 Model: "TextClassifier(
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
               

2020-10-22 01:59:06,969 ----------------------------------------------------------------------------------------------------
2020-10-22 01:59:06,969 Corpus: "Corpus: 11762 train + 3911 dev + 3906 test sentences"
2020-10-22 01:59:06,970 ----------------------------------------------------------------------------------------------------
2020-10-22 01:59:06,970 Parameters:
2020-10-22 01:59:06,970  - learning_rate: "3e-05"
2020-10-22 01:59:06,971  - mini_batch_size: "1"
2020-10-22 01:59:06,971  - patience: "3"
2020-10-22 01:59:06,971  - anneal_factor: "0.5"
2020-10-22 01:59:06,972  - max_epochs: "50"
2020-10-22 01:59:06,972  - shuffle: "True"
2020-10-22 01:59:06,973  - train_with_dev: "False"
2020-10-22 01:59:06,973  - batch_growth_annealing: "False"
2020-10-22 01:59:06,973 ----------------------------------------------------------------------------------------------------
2020-10-22 01:59:06,974 Model training base path: "classifiers/spooky_authorship_classifier_transformer"
2020-10-22 01

2020-10-22 03:08:01,902 epoch 5 - iter 10584/11762 - loss 0.16065545 - samples/sec: 19.66 - lr: 0.000030
2020-10-22 03:09:03,078 epoch 5 - iter 11760/11762 - loss 0.16426696 - samples/sec: 19.28 - lr: 0.000030
2020-10-22 03:09:03,214 ----------------------------------------------------------------------------------------------------
2020-10-22 03:09:03,215 EPOCH 5 done: loss 0.1643 - lr 0.0000300
2020-10-22 03:09:33,793 DEV : loss 1.2200660705566406 - score 0.8312
2020-10-22 03:09:35,671 BAD EPOCHS (no improvement): 2
2020-10-22 03:09:35,672 ----------------------------------------------------------------------------------------------------
2020-10-22 03:10:37,064 epoch 6 - iter 1176/11762 - loss 0.13672677 - samples/sec: 19.23 - lr: 0.000030
2020-10-22 03:11:38,217 epoch 6 - iter 2352/11762 - loss 0.13037400 - samples/sec: 19.29 - lr: 0.000030
2020-10-22 03:12:38,929 epoch 6 - iter 3528/11762 - loss 0.12928274 - samples/sec: 19.43 - lr: 0.000030
2020-10-22 03:13:39,718 epoch 6 - iter 

2020-10-22 04:05:06,359 epoch 11 - iter 2352/11762 - loss 0.48739608 - samples/sec: 19.49 - lr: 0.000015
2020-10-22 04:06:07,073 epoch 11 - iter 3528/11762 - loss 0.45137204 - samples/sec: 19.42 - lr: 0.000015
2020-10-22 04:07:07,693 epoch 11 - iter 4704/11762 - loss 0.43779539 - samples/sec: 19.46 - lr: 0.000015
2020-10-22 04:08:08,821 epoch 11 - iter 5880/11762 - loss 0.48126263 - samples/sec: 19.29 - lr: 0.000015
2020-10-22 04:09:09,987 epoch 11 - iter 7056/11762 - loss 0.49364693 - samples/sec: 19.28 - lr: 0.000015
2020-10-22 04:10:11,475 epoch 11 - iter 8232/11762 - loss 0.52547924 - samples/sec: 19.18 - lr: 0.000015
2020-10-22 04:11:12,445 epoch 11 - iter 9408/11762 - loss 0.57372096 - samples/sec: 19.35 - lr: 0.000015
2020-10-22 04:12:16,188 epoch 11 - iter 10584/11762 - loss 0.61729554 - samples/sec: 18.50 - lr: 0.000015
2020-10-22 04:13:18,869 epoch 11 - iter 11760/11762 - loss 0.65836284 - samples/sec: 18.82 - lr: 0.000015
2020-10-22 04:13:19,013 -----------------------------

2020-10-22 05:04:38,705 epoch 16 - iter 9408/11762 - loss 0.24334196 - samples/sec: 19.45 - lr: 0.000004
2020-10-22 05:05:37,681 epoch 16 - iter 10584/11762 - loss 0.24516871 - samples/sec: 20.00 - lr: 0.000004
2020-10-22 05:06:38,146 epoch 16 - iter 11760/11762 - loss 0.24826148 - samples/sec: 19.51 - lr: 0.000004
2020-10-22 05:06:38,285 ----------------------------------------------------------------------------------------------------
2020-10-22 05:06:38,286 EPOCH 16 done: loss 0.2482 - lr 0.0000038
2020-10-22 05:07:09,102 DEV : loss 1.1977378129959106 - score 0.7934
2020-10-22 05:07:11,084 BAD EPOCHS (no improvement): 1
2020-10-22 05:07:11,085 ----------------------------------------------------------------------------------------------------
2020-10-22 05:08:11,359 epoch 17 - iter 1176/11762 - loss 0.13938726 - samples/sec: 19.58 - lr: 0.000004
2020-10-22 05:09:12,185 epoch 17 - iter 2352/11762 - loss 0.19535208 - samples/sec: 19.39 - lr: 0.000004
2020-10-22 05:10:13,258 epoch 17 

{'test_score': 0.8469,
 'dev_score_history': [0.8251,
  0.8417,
  0.8468,
  0.8397,
  0.8312,
  0.8146,
  0.7366,
  0.7977,
  0.7635,
  0.5285,
  0.4017,
  0.7891,
  0.7233,
  0.8006,
  0.8105,
  0.7934,
  0.8131,
  0.8018,
  0.822],
 'train_loss_history': [0.8474903985428051,
  0.4328962669024286,
  0.2870155546113206,
  0.20389938118258502,
  0.1642565741552335,
  0.17634345009699567,
  0.31582380232078633,
  0.6961090544328526,
  0.43322091353308045,
  0.5079981821086628,
  0.658336912656301,
  0.7135963585032439,
  0.39383146148178166,
  0.34309273786842465,
  0.30168497799569527,
  0.24822078653878338,
  0.20477457962662532,
  0.1566602155592032,
  0.12913906098645614],
 'dev_loss_history': [0.6951817274093628,
  0.9179255962371826,
  0.6853742003440857,
  0.9877879619598389,
  1.2200660705566406,
  1.2726454734802246,
  1.0422911643981934,
  1.0451858043670654,
  1.0180667638778687,
  2.696859836578369,
  1.2079285383224487,
  1.0325223207473755,
  0.9829927086830139,
  1.1076633